In [ ]:
import pickle
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.signal import iirnotch, lfilter
import os


In [ ]:
def plot_frames(original_data, normalized_data, title, num_frames=10, num_channels=8):
    keys = list(original_data.keys())[:num_frames] 
    num_plots = len(keys) * num_channels
    
    fig, axs = plt.subplots(num_plots, 2, figsize=(15, 2*num_plots))
    fig.suptitle(title)
    
    for i, key in enumerate(keys):
        for ch in range(num_channels):
            idx = i * num_channels + ch
            axs[idx, 0].plot(original_data[key][:, ch])
            if ch == 0:
                axs[idx, 0].set_title(f'Frame {key} - Before Normalization')
            axs[idx, 0].set_ylabel(f'Ch {ch+1}')
            
            axs[idx, 1].plot(normalized_data[key][:, ch])
            if ch == 0:
                axs[idx, 1].set_title(f'Frame {key} - After Normalization')
            axs[idx, 1].set_ylabel(f'Ch {ch+1}')
    
    for ax in axs.flat:
        ax.set_xlabel('Sample')
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
def normalize_data(data):
    normalized_data = {}
    emg_frame_numbers = sorted([key for key in data.keys() if not np.isnan(key)])
    print(f"Found {len(emg_frame_numbers)} EMG frames")
    print(f"Patient has {len(data[1001][0])} channels for EMG data")
    print()

    channels = [[] for _ in range(8)]
    for key in emg_frame_numbers:
        for ch in range(8):  
            channels[ch].extend(data[key][ch])

    for ch in range(8):
        min_val = min(channels[ch])
        max_val = max(channels[ch])
        normalized_values = [(value - min_val) / (max_val - min_val) if max_val > min_val else 0 for value in channels[ch]]

        index = 0
        for key in emg_frame_numbers:
            frame_length = len(data[key][ch])
            normalized_data.setdefault(key, [[] for _ in range(8)])
            normalized_data[key][ch] = normalized_values[index:index+frame_length]
            index += frame_length

    # emg_data_arrays = {key: np.array(value).T for key, value in data.items()} 
    # eeg_data_arrays = {key: np.array(value).T for key, value in data.items()} 

    # print(f"Shape of EMG data: {list(emg_data_arrays.values())[0].shape}")
    # print(f"Shape of EEG data: {list(eeg_data_arrays.values())[0].shape}")

    # print("Len emg" , len(emg_data_arrays.keys()))
    # print("Len eeg" , len(eeg_data_arrays.keys()))

    # normalized_emg_data_arrays = {key: np.array(value).T for key, value in normalized_data.items()}  
    # normalized_eeg_data_arrays = {key: np.array(value).T for key, value in normalized_data.items()}  

    # print(f"Shape of normalized EMG data: {list(normalized_emg_data_arrays.values())[0].shape}")
    # print(f"Shape of normalized EEG data: {list(normalized_eeg_data_arrays.values())[0].shape}")

    # print("Len normalized emg" , len(normalized_emg_data_arrays.keys()))
    # print("Len normalized eeg" , len(normalized_eeg_data_arrays.keys()))
    # plot_frames(eeg_data_arrays, normalized_eeg_data_arrays, 'EMG Data Normalization')        

    return normalized_data


In [ ]:
total_frames = 0
dir = 'C:\\Users\\MindRove_BZs\\Diploma\\eeg_emg\\'
output_dir = 'C:\\Users\\MindRove_BZs\\Diploma\\processed_data\\'  

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

fs = 500  
f0 = 50  
Q = 10 

b, a = iirnotch(f0, Q, fs)
for i in range(5, 6):
    with open(dir + f'emg_data_patient_{i}.pkl', 'rb') as f:
        emg_data = pickle.load(f)
    emg_frame_numbers = sorted([key for key in emg_data.keys() if not np.isnan(key)])
    with open(dir + f'eeg_data_patient_{i}.pkl', 'rb') as f:
        eeg_data = pickle.load(f)
    eeg_frame_numbers = sorted(eeg_data.keys())
    print(f'Patient {i}: {len(emg_frame_numbers)} EMG frames, {len(eeg_frame_numbers)} EEG frames')
    
    normalized_emg_data = {}
    normalized_eeg_data = {}
    normalized_data_eeg = normalize_data(eeg_data)
    normalize_data_emg = normalize_data(emg_data)

    for key in emg_frame_numbers:
        filtered_data = lfilter(b, a, normalize_data_emg[key], axis=0)
        normalized_emg_data[key] = filtered_data
    
    for key in eeg_frame_numbers:
        filtered_data = lfilter(b, a, normalized_data_eeg[key], axis=0)
        normalized_eeg_data[key] =  filtered_data
    
    print(f'Patient {i}: Normalized and filtered EMG and EEG data shapes: {len(normalized_emg_data)}, {len(normalized_eeg_data)}')
    
    with open(output_dir + f'normalized_filtered_emg_data_patient_{i}.pkl', 'wb') as f:
        pickle.dump(normalized_emg_data, f)
    with open(output_dir + f'normalized_filtered_eeg_data_patient_{i}.pkl', 'wb') as f:
        pickle.dump(normalized_eeg_data, f)

In [ ]:
emg_data_arrays = {key: np.array(value).T for key, value in emg_data.items()}  
eeg_data_arrays = {key: np.array(value).T for key, value in eeg_data.items()}  

print(f"Shape of EMG data: {list(emg_data_arrays.values())[0].shape}")
print(f"Shape of EEG data: {list(eeg_data_arrays.values())[0].shape}")
print("Len emg" , len(emg_data_arrays.keys()))
print("Len eeg" , len(eeg_data_arrays.keys()))

normalized_emg_data_arrays = {key: np.array(value).T for key, value in normalized_emg_data.items()}  
normalized_eeg_data_arrays = {key: np.array(value).T for key, value in normalized_eeg_data.items()} 

print(f"Shape of normalized EMG data: {list(normalized_emg_data_arrays.values())[0].shape}")
print(f"Shape of normalized EEG data: {list(normalized_eeg_data_arrays.values())[0].shape}")
print("Len normalized emg" , len(normalized_emg_data_arrays.keys()))
print("Len normalized eeg" , len(normalized_eeg_data_arrays.keys()))

In [ ]:
plot_frames(emg_data_arrays, normalized_emg_data_arrays, 'EMG Data Before and After Normalization')

In [ ]:

def plot_combined_frames_with_normalized(original_data, normalized_data, title, num_frames=5, num_channels=8):
    keys = list(original_data.keys())[:num_frames]  
    
    fig, axs = plt.subplots(num_channels, 2, figsize=(30, 2*num_channels), sharex='col')
    fig.suptitle(title)
    
    for ch in range(num_channels):
        combined_data_original = []
        combined_data_normalized = []
        for key in keys:
            combined_data_original.extend(original_data[key][:, ch])
            combined_data_normalized.extend(normalized_data[key][:, ch])
        
        axs[ch, 0].plot(combined_data_original)
        axs[ch, 0].set_ylabel(f'Ch {ch+1}')
        axs[ch, 0].set_title(f'Original Channel {ch+1}', loc='left')
        axs[ch, 1].plot(combined_data_normalized)
        axs[ch, 1].set_title(f'Filtered and normalized Channel {ch+1}', loc='left')
    
    axs[-1, 0].set_xlabel('Sample')
    axs[-1, 1].set_xlabel('Sample')
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
plot_combined_frames_with_normalized(emg_data_arrays, normalized_emg_data_arrays, 'EMG Data Before and After preprocessing')